In [59]:
import pandas as pd
from datetime import datetime

In [60]:
TWEETS_812A_JAN = "data/tweets/812A_2019_JAN_2KM.csv"
TWEETS_812A_FEV = "data/tweets/812A_2019_FEV_2KM.csv"
TWEETS_812A_MAR = "data/tweets/812A_2019_MAR_2KM.csv"
TWEETS_833A = "data/tweets/833A_2019_2KM.csv"

In [61]:
RAIN_GAUGE_CODES = ["355030812A", "355030833A"]
RAIN_GAUGES_JAN = "data/rain_gauges/data1.csv"
RAIN_GAUGES_FEV = "data/rain_gauges/data2.csv"
RAIN_GAUGES_MAR = "data/rain_gauges/data3.csv"

In [62]:
KEYWORDS = ["chuva", "chove", "chuvoso", "chuvosa", "chuvarada", 
            "alagamento", "alagou", "inundação", "inundada", "enxurrada",
            "rio transbordou", "córrego transbordou", "rio saiu da calha", 
            "córrego saiu da calha", "deslizamento", "escorregamento", "queda de barreira"]

## Time range creation

In [63]:
DAYS_RANGE = datetime(2019, 4, 1) - datetime(2019, 1, 1) 
TIME_PERIOD = DAYS_RANGE.days * 24 * 6

In [64]:
begin_date = datetime.strptime("01-01-2019", "%d-%m-%Y")

In [65]:
date_range = pd.date_range(begin_date, periods=TIME_PERIOD, freq="10min")

In [66]:
date_range = date_range.to_frame(index=False, name="time")

## Rain Gauge Data

In [67]:
rain_gauges_jan = pd.read_csv(RAIN_GAUGES_JAN, sep=";", index_col=False, decimal=".")
rain_gauges_fev = pd.read_csv(RAIN_GAUGES_FEV, sep=";", index_col=False)
rain_gauges_mar = pd.read_csv(RAIN_GAUGES_MAR, sep=";", index_col=False)

In [68]:
rain_gauges = pd.concat([rain_gauges_jan, rain_gauges_fev, rain_gauges_mar])

In [69]:
filtered_rain_gauges = rain_gauges[rain_gauges["codEstacao"].isin(RAIN_GAUGE_CODES)]

In [70]:
filtered_rain_gauges["datahora"] = pd.to_datetime(filtered_rain_gauges["datahora"])
filtered_rain_gauges["valorMedida"] = filtered_rain_gauges["valorMedida"].str.replace(",", ".")
filtered_rain_gauges["valorMedida"] = pd.to_numeric(filtered_rain_gauges["valorMedida"])

C:\Users\Jefferson\AppData\Local\Temp\ipykernel_10848\3252865042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rain_gauges["datahora"] = pd.to_datetime(filtered_rain_gauges["datahora"])
C:\Users\Jefferson\AppData\Local\Temp\ipykernel_10848\3252865042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rain_gauges["valorMedida"] = filtered_rain_gauges["valorMedida"].str.replace(",", ".")
C:\Users\Jefferson\AppData\Local\Temp\ipykernel_10848\3252865042.py:3: SettingWithCopyWarning: 
A 

In [71]:
filtered_rain_gauges = filtered_rain_gauges[["codEstacao", "datahora", "valorMedida"]]

In [72]:
rain_gauge_812A = filtered_rain_gauges[filtered_rain_gauges["codEstacao"] == "355030812A"]

In [73]:
rain_gauge_833A = filtered_rain_gauges[filtered_rain_gauges["codEstacao"] == "355030833A"]

In [74]:
time_series = pd.merge(date_range, rain_gauge_812A, left_on="time", right_on="datahora", how="left")

In [75]:
time_series = time_series.drop(columns=["codEstacao", "datahora"])
time_series = time_series.rename(columns={"valorMedida": "812A"})
time_series["812A"] = time_series["812A"].fillna(0)

In [76]:
time_series = pd.merge(time_series, rain_gauge_833A, left_on="time", right_on="datahora", how="left")

In [77]:
time_series = time_series.drop(columns=["codEstacao", "datahora"])
time_series = time_series.rename(columns={"valorMedida": "833A"})
time_series["833A"] = time_series["833A"].fillna(0)

In [85]:
time_series.head()

,time,812A,833A
0,2019-01-01 00:00:00,0.0,0.0
1,2019-01-01 00:10:00,0.0,0.0
2,2019-01-01 00:20:00,0.0,0.0
3,2019-01-01 00:30:00,0.0,0.0
4,2019-01-01 00:40:00,0.0,0.0


## Tweet data

In [79]:
tweets_812A_jan = pd.read_csv(TWEETS_812A_JAN)
tweets_812A_fev = pd.read_csv(TWEETS_812A_FEV)
tweets_812_mar = pd.read_csv(TWEETS_812A_MAR)
tweets_812 = pd.concat([tweets_812A_jan, tweets_812A_fev, tweets_812_mar])

In [80]:
tweets_833 = pd.read_csv(TWEETS_833A)

In [81]:
tweets = pd.concat([tweets_812, tweets_833])

In [82]:
tweet_data = tweets[["date", "text"]]

In [83]:
pattern = "|".join(KEYWORDS)

In [84]:
filtered_data = tweet_data[tweet_data["text"].str.contains(pattern, case=False, na=False)]

In [93]:
filtered_data.loc[:, "date"] = pd.to_datetime(filtered_data["date"])

In [92]:
filtered_data.loc[:, "date"] = filtered_data["date"].dt.round("10min")

In [97]:
tweet_count = filtered_data.groupby('date').size().reset_index(name='tweet_count')

In [110]:
total_data = pd.merge(time_series, tweet_count, left_on="time", right_on="date", how="left")

In [111]:
total_data = total_data.drop(columns="date")

In [115]:
total_data["tweet_count"] = total_data["tweet_count"].fillna(0)
total_data["tweet_count"] = total_data["tweet_count"].astype(int)

In [119]:
total_data.to_csv("data/results/time_data.csv", sep=";")